# iiif

> IIIF Client

In [ ]:
#| default_exp iiif

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import json
import os
from google.cloud import vision
from google.oauth2 import service_account
from PIL import Image
import io
from google.cloud.vision import AnnotateImageResponse
import shutil

In [ ]:
#| export
class IIIFClient:

    ID_PREFIX = "http://example.org/iiif/abc"
    ANNOTATION_LABEL = "GOCR"

    def create_manifest3_by_gocr(self, url: str, cred_file_path: str, output_iiif_manifest_file_path: str, tmp_dir: str = "./tmp", debug=False) -> None:
        """
        URLから画像をダウンロードし、GOCR APIを使用してManifest3ファイルを作成する。

        Parameters
        ----------
        url : str
            画像のURL
        cred_file_path : str
            GOCR APIの認証情報ファイルのパス
        output_iiif_manifest_file_path : str
            出力するIIIFマニフェストファイルのパス
        tmp_dir : str
            一時ディレクトリのパス
        debug : bool
            デバッグモードの有無

        Returns
        -------
        None
        """
        tmp_img_path = self._download_image(url, tmp_dir)

        if tmp_img_path is not None:
            self._create_manifest3_local_by_gocr(tmp_img_path, cred_file_path, output_iiif_manifest_file_path, debug=debug)

    def _download_image(self, url: str, tmp_dir: str) -> str:
        """
        画像をURLからダウンロードし、一時ディレクトリに保存する。
        """
        tmp_img_path = os.path.join(tmp_dir, "01/images", url.replace("/", "%2F"))
        if not os.path.exists(tmp_img_path):
            os.makedirs(os.path.dirname(tmp_img_path), exist_ok=True)
            try:
                headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
                response = requests.get(url, headers=headers)
                # response = requests.get(url)
                response.raise_for_status()  # ステータスコードが200番台でない場合にエラーを発生させる
                with open(tmp_img_path, 'wb') as f:
                    f.write(response.content)

                # ダウンロードしたファイルが画像として認識されるか確認する
                Image.open(tmp_img_path).verify()
            except requests.RequestException as e:
                print(f"リクエスト中にエラーが発生しました: {e}")
                return None
            except IOError:
                print(f"ダウンロードされたファイルが画像として認識できません: {tmp_img_path}")
                return None
        return tmp_img_path
    
    def _create_manifest3_local_by_gocr(self, tmp_img_path: str, api_key: str, output_path: str, debug=False) -> None:
        """
        ローカルに保存された画像からGOCR APIを使用してManifest3ファイルを作成する。
        """
        img_width, img_height = self.get_image_dimensions(tmp_img_path)
        json_data = self.perform_ocr(tmp_img_path, api_key)
        if debug:
            print(json.dumps(json_data, indent=4, ensure_ascii=False))
        manifest3 = self.create_manifest3_data(json_data, img_width, img_height)
        self.write_manifest3_file(manifest3, output_path)
        self.copy_image_to_output(tmp_img_path, output_path)

    def get_image_dimensions(self, image_path: str) -> tuple:
        """
        画像の寸法を取得する。
        """
        with Image.open(image_path) as im:
            return im.size
        
    def perform_ocr(self, image_path: str, api_key: str) -> dict:
        """
        Google Cloud Vision APIを使用してOCRを実行し、結果をJSONとして返す。
        """
        credentials = service_account.Credentials.from_service_account_file(api_key)
        client = vision.ImageAnnotatorClient(credentials=credentials)
        with io.open(image_path, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)
        response = client.text_detection(image=image)
        json_str = AnnotateImageResponse.to_json(response)
        return json.loads(json_str)
    
    def create_manifest3_data(self, json_data: dict, img_width: int, img_height: int) -> dict:
        """
        OCR結果からManifest3データを作成する。
        """

        annotations = []

        annotations_ = json_data["textAnnotations"][1:]

        id_prefix = self.ID_PREFIX
        label = self.ANNOTATION_LABEL

        for annotation_ in annotations_:

            min_x = float('inf')
            min_y = float('inf')
            max_x = float('-inf')
            max_y = float('-inf')

            # 全ての頂点をループして最小値と最大値を見つける
            for vertex in annotation_["boundingPoly"]["vertices"]:
                x = vertex["x"]
                y = vertex["y"]
                min_x = min(min_x, x)
                min_y = min(min_y, y)
                max_x = max(max_x, x)
                max_y = max(max_y, y)

            # 幅と高さを計算
            width = max_x - min_x
            height = max_y - min_y

            xywh = (min_x, min_y, width, height)

            xywh = ','.join(map(str, xywh))

            annotation = {
                "body": {
                    "type": "TextualBody",
                    "value": f'[{str(len(annotations) + 1).zfill(5)}] {annotation_["description"]}',
                    "format": "text/plain"
                },
                "id": f"{id_prefix}/canvas/p2#xywh={xywh.replace(',', '/')}",
                "type": "Annotation",
                "motivation": "commenting",
                "target": f"{id_prefix}/canvas/p2#xywh={xywh}"
            }
            annotations.append(annotation)

        manifest3 = {
            "@context": "http://iiif.io/api/presentation/3/context.json",
            "id": f"{id_prefix}/manifest",
            "label": {
                "none": [
                    label
                ]
            },
            "type": "Manifest",
            "items": [
                {
                    "id": f"{id_prefix}/canvas/p2",
                    "type": "Canvas",
                    "label": {
                        "none": [
                            label
                        ]
                    },
                    "height": img_height,
                    "width": img_width,
                    "items": [
                        {
                        "id": f"{id_prefix}/annotation/p0002-image",
                        "type": "AnnotationPage",
                        "items": [
                            {
                            "body": {
                                "id": f"images/{'demo.jpg'}", # os.path.basename(tmp_img_path)
                                "type": "Image",
                                "format": "image/jpeg",
                                "height": img_height,
                                "width": img_width,
                            },
                            "id": f"{id_prefix}/annotation/p0002-image/anno",
                            "type": "Annotation",
                            "motivation": "painting",
                            "target": f"{id_prefix}/canvas/p2"
                            }
                        ]
                        }
                    ],
                    "annotations": [
                        {
                            "id": f"{id_prefix}/canvas/p2/curation",
                            "type": "AnnotationPage",
                            "items": annotations
                        }
                    ]
                }
            ]
        }

        return manifest3
    
    def write_manifest3_file(self, manifest3: dict, output_path: str) -> None:
        """
        Manifest3データをファイルに書き込む。
        """
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with open(output_path, 'wt') as f:
            json.dump(manifest3, f, indent=4, ensure_ascii=False)

    def copy_image_to_output(self, image_path: str, output_path: str) -> None:
        """
        出力ディレクトリに画像をコピーする。
        """
        output_image_path = os.path.join(os.path.dirname(output_path), "images", "demo.jpg")
        os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

        shutil.copyfile(image_path, output_image_path)
        

In [ ]:
show_doc(IIIFClient.create_manifest3_by_gocr)

---

### IIIFClient.create_manifest3_by_gocr

>      IIIFClient.create_manifest3_by_gocr (url:str, cred_file_path:str,
>                                           output_iiif_manifest_file_path:str,
>                                           tmp_dir:str='./tmp', debug=False)

URLから画像をダウンロードし、GOCR APIを使用してManifest3ファイルを作成する。

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| url | str |  | 画像のURL |
| cred_file_path | str |  | GOCR APIの認証情報ファイルのパス |
| output_iiif_manifest_file_path | str |  | 出力するIIIFマニフェストファイルのパス |
| tmp_dir | str | ./tmp | 一時ディレクトリのパス |
| debug | bool | False | デバッグモードの有無 |
| **Returns** | **None** |  |  |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()